# Initial Setup
Download and extract dataset. Upload previously saved model weights. Execute only once.

In [0]:
# check if data directory is present
!ls -l

In [0]:
# upload previously saved model weights 
from google.colab import files
uploaded = files.upload()

# save as file into current working directory
with open("model-1.h5", "wb") as f:
  f.write(uploaded["model-1.h5"])

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
# download data files and labels
# drive.CreateFile({'id': '1kkOgZp7aul6v-9K7H11uxOHM7f21VUPi'}).GetContentFile('stage1_sample_submission.csv.zip')
drive.CreateFile({'id': '1KvOWg2QolRvaiAmFLR5nJ3n7sP5-i_SL'}).GetContentFile('stage1_test.zip')
drive.CreateFile({'id': '1kkOgZp7aul6v-9K7H11uxOHM7f21VUPi'}).GetContentFile('stage1_train_labels.csv.zip')
drive.CreateFile({'id': '1wnZMeMNn-tZrSdFGCS4QFKN8dm3Tf5nS'}).GetContentFile('stage1_train.zip')

In [0]:
# create data directories
!mkdir -p data/test
!mkdir -p data/train

In [0]:
# unzip files
!unzip -qq -o stage1_test.zip -d data/test
!unzip -qq -o stage1_train.zip -d data/train
!unzip -qq -o stage1_train_labels.csv.zip -d data
# !unzip -qq -o stage1_sample_submission.csv.zip -d data

# Define model and visualize data

In [0]:
!pip install tensorflow-gpu==1.4.1 keras tqdm

Adapted U-net implementation from Kjetil Åmdal-Sævik: https://www.kaggle.com/keegil/keras-u-net-starter-lb-0-277

In [0]:
# show available devices
# from tensorflow.python.client import device_lib
# device_lib.list_local_devices()

In [0]:
import os
import sys
import random
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from tqdm import tqdm
from itertools import chain
from skimage.io import imread, imshow, imread_collection, concatenate_images
from skimage.transform import resize
from skimage.morphology import label

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
from keras import backend as K
from keras.layers.advanced_activations import LeakyReLU


import tensorflow as tf

# Set some parameters
IMG_WIDTH = 256  # width/height in original script was 128
IMG_HEIGHT = 256
IMG_CHANNELS = 3
TRAIN_PATH = 'data/train/'
TEST_PATH = 'data/test/'

warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42

In [0]:
# Get train and test IDs
train_ids = next(os.walk(TRAIN_PATH))[1]
test_ids = next(os.walk(TEST_PATH))[1]

Import the images and associated masks and rescale to 256x256.

In [0]:
# Get and resize train images and masks
X_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
Y_train = np.zeros((len(train_ids), IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
print('Getting and resizing train images and masks ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(train_ids), total=len(train_ids)):
    path = TRAIN_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_train[n] = img
    mask = np.zeros((IMG_HEIGHT, IMG_WIDTH, 1), dtype=np.bool)
    for mask_file in next(os.walk(path + '/masks/'))[2]:
        mask_ = imread(path + '/masks/' + mask_file)
        mask_ = np.expand_dims(resize(mask_, (IMG_HEIGHT, IMG_WIDTH), mode='constant', 
                                      preserve_range=True), axis=-1)
        mask = np.maximum(mask, mask_)
    Y_train[n] = mask

# Get and resize test images
X_test = np.zeros((len(test_ids), IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS), dtype=np.uint8)
sizes_test = []
print('Getting and resizing test images ... ')
sys.stdout.flush()
for n, id_ in tqdm(enumerate(test_ids), total=len(test_ids)):
    path = TEST_PATH + id_
    img = imread(path + '/images/' + id_ + '.png')[:,:,:IMG_CHANNELS]
    sizes_test.append([img.shape[0], img.shape[1]])
    img = resize(img, (IMG_HEIGHT, IMG_WIDTH), mode='constant', preserve_range=True)
    X_test[n] = img

print('Done!')

In [0]:
# Display random sample data
ix = random.randint(0, len(train_ids))
imshow(X_train[ix])
plt.show()
imshow(np.squeeze(Y_train[ix]))
plt.show()

## Define Keras metrics
Try different metrics:
* Intersection over union (IoU, or Jaccard index) https://en.wikipedia.org/wiki/Jaccard_index
* Dice coefficient https://en.wikipedia.org/wiki/S%C3%B8rensen%E2%80%93Dice_coefficient

In [0]:
# Define IoU metric
def mean_iou(y_true, y_pred):
    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        y_pred_ = tf.to_int32(y_pred > t)
        score, up_opt = tf.metrics.mean_iou(y_true, y_pred_, 2)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([up_opt]):
            score = tf.identity(score)
        prec.append(score)
    return K.mean(K.stack(prec), axis=0)

In [0]:
# we are not using dice coeffienct metric

# dice coefficient implementation from
# https://github.com/jocicmarko/ultrasound-nerve-segmentation

# dice coefficient metric
smooth = 1.

def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)

## Define neural network
U-Net model, loosely based on U-Net: Convolutional Networks for Biomedical Image Segmentation https://arxiv.org/pdf/1505.04597.pdf and very similar to https://github.com/jocicmarko/ultrasound-nerve-segmentation

In [0]:
# Build U-Net model
inputs = Input((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
s = Lambda(lambda x: x / 255) (inputs)

c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (s)
c1 = BatchNormalization(axis=1) (c1)
c1 = Dropout(0.1) (c1)
c1 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c1)
c1 = BatchNormalization(axis=1) (c1)
p1 = MaxPooling2D((2, 2)) (c1)

c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p1)
c2 = BatchNormalization(axis=1) (c2)
c2 = Dropout(0.1) (c2)
c2 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c2)
c2 = BatchNormalization(axis=1) (c2)
p2 = MaxPooling2D((2, 2)) (c2)

c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p2)
c3 = BatchNormalization(axis=1) (c3)
c3 = Dropout(0.2) (c3)
c3 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c3)
c3 = BatchNormalization(axis=1) (c3)
p3 = MaxPooling2D((2, 2)) (c3)

c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p3)
c4 = BatchNormalization(axis=1) (c4)
c4 = Dropout(0.2) (c4)
c4 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c4)
c4 = BatchNormalization(axis=1) (c4)
p4 = MaxPooling2D(pool_size=(2, 2)) (c4)

c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (p4)
c5 = BatchNormalization(axis=1) (c5)
c5 = Dropout(0.3) (c5)
c5 = Conv2D(256, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c5)
c5 = BatchNormalization(axis=1) (c5)

u6 = Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same') (c5)
u6 = concatenate([u6, c4])
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u6)
c6 = BatchNormalization(axis=1) (c6)
c6 = Dropout(0.2) (c6)
c6 = Conv2D(128, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c6)
c6 = BatchNormalization(axis=1) (c6)

u7 = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same') (c6)
u7 = concatenate([u7, c3])
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u7)
c7 = BatchNormalization(axis=1) (c7)
c7 = Dropout(0.2) (c7)
c7 = Conv2D(64, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c7)

u8 = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same') (c7)
u8 = concatenate([u8, c2])
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u8)
c8 = BatchNormalization(axis=1) (c8)
c8 = Dropout(0.1) (c8)
c8 = Conv2D(32, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c8)

u9 = Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same') (c8)
u9 = concatenate([u9, c1], axis=3)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (u9)
c9 = BatchNormalization(axis=1) (c9)
c9 = Dropout(0.1) (c9)
c9 = Conv2D(16, (3, 3), activation='elu', kernel_initializer='he_normal', padding='same') (c9)

outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)

model = Model(inputs=[inputs], outputs=[outputs])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[mean_iou])

In [0]:
# model.summary()

# Data augmentation
Imagedatagenerator documentation at https://keras.io/preprocessing/image/
* keras augmentation implementation from https://www.kaggle.com/c0conuts/unet-imagedatagenerator-lb-0-336
* elastic transforms implementation from https://www.kaggle.com/ori226/data-augmentation-with-elastic-deformations


## Elastic transform

In [0]:
# install and import opencv
# !apt-get -qq install -y libsm6 libxext6 && pip install -q -U opencv-python

In [0]:
# Function to distort image
# from scipy.ndimage.interpolation import map_coordinates
# from scipy.ndimage.filters import gaussian_filter

# # Function to distort image
# def elastic_transform(image, alpha, sigma, alpha_affine, random_state=None):
#     """Elastic deformation of images as described in [Simard2003]_ (with modifications).
#     .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
#          Convolutional Neural Networks applied to Visual Document Analysis", in
#          Proc. of the International Conference on Document Analysis and
#          Recognition, 2003.

#      Based on https://gist.github.com/erniejunior/601cdf56d2b424757de5
#     """
#     if random_state is None:
#         random_state = np.random.RandomState(None)

#     shape = image.shape
#     shape_size = shape[:2]
    
#     # Random affine
#     center_square = np.float32(shape_size) // 2
#     square_size = min(shape_size) // 3
#     pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
#     pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
#     M = cv2.getAffineTransform(pts1, pts2)
#     image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)

#     dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
#     dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma) * alpha
#     dz = np.zeros_like(dx)

#     x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
#     indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

#     return map_coordinates(image, indices, order=1, mode='reflect').reshape(shape)

Visualize elastic distortions.

In [0]:
# Load images
#im = x.next()[0].astype(np.uint8)
#im_mask = y.next()[0].astype(np.uint8)

In [0]:
# BATCH_SIZE=48

In [0]:
# # Define function to draw a grid
# def draw_grid(im, grid_size):
#     # Draw grid lines
#     for i in range(0, im.shape[1], grid_size):
#         cv2.line(im, (i, 0), (i, im.shape[0]), color=(255,))
# #     for j in range(0, im.shape[0], grid_size):
#         cv2.line(im, (0, j), (im.shape[1], j), color=(255,))

# # Load images
# im = cv2.imread("../input/train/10_1.tif", -1)
# # im_mask = cv2.imread("../input/train/10_1_mask.tif", -1)

# # Draw grid lines
# draw_grid(im, 50)
# draw_grid(im_mask, 50)

# # Merge images into separete channels (shape will be (cols, rols, 2))
# im_merge = np.concatenate((im[...,None], im_mask[...,None]), axis=2)

In [0]:
# # First sample...

# %matplotlib inline

# # Apply transformation on image
# im_merge_t = elastic_transform(im_merge, im_merge.shape[1] * 2, im_merge.shape[1] * 0.08, im_merge.shape[1] * 0.08)

# # Split image and mask
# im_t = im_merge_t[...,0]
# im_mask_t = im_merge_t[...,1]

# # Display result
# plt.figure(figsize = (16,14))
# plt.imshow(np.c_[np.r_[im, im_mask], np.r_[im_t, im_mask_t]], cmap='gray')

In [0]:
# Second sample (heavyer transform)...

# %matplotlib inline

# # Apply transformation on image
# im_merge_t = elastic_transform(im_merge, im_merge.shape[1] * 3, im_merge.shape[1] * 0.07, im_merge.shape[1] * 0.09)

# # Split image and mask
# im_t = im_merge_t[...,0]
# im_mask_t = im_merge_t[...,1]

# # Display result
# plt.figure(figsize = (16,14))
# plt.imshow(np.c_[np.r_[im, im_mask], np.r_[im_t, im_mask_t]], cmap='gray')

## Create Imagedatagenerator

In [0]:
BATCH_SIZE = 64

In [0]:
from keras.preprocessing import image

# Creating the training Image and Mask generator
image_datagen = image.ImageDataGenerator(
                                         shear_range=0.5, rotation_range=50, 
                                         zoom_range=0.2, width_shift_range=0.2, 
                                         height_shift_range=0.2, fill_mode='reflect')
mask_datagen = image.ImageDataGenerator(
                                        shear_range=0.5, rotation_range=50, 
                                        zoom_range=0.2, width_shift_range=0.2, 
                                        height_shift_range=0.2, fill_mode='reflect')

# Keep the same seed for image and mask generators so they fit together
image_datagen.fit(X_train[:int(X_train.shape[0]*0.9)], augment=True, seed=seed)
mask_datagen.fit(Y_train[:int(Y_train.shape[0]*0.9)], augment=True, seed=seed)

x=image_datagen.flow(X_train[:int(X_train.shape[0]*0.9)],batch_size=BATCH_SIZE,shuffle=True, seed=seed)
y=mask_datagen.flow(Y_train[:int(Y_train.shape[0]*0.9)],batch_size=BATCH_SIZE,shuffle=True, seed=seed)


# Creating the validation Image and Mask generator
image_datagen_val = image.ImageDataGenerator()
mask_datagen_val = image.ImageDataGenerator()

image_datagen_val.fit(X_train[int(X_train.shape[0]*0.9):], augment=True, seed=seed)
mask_datagen_val.fit(Y_train[int(Y_train.shape[0]*0.9):], augment=True, seed=seed)

x_val=image_datagen_val.flow(X_train[int(X_train.shape[0]*0.9):],batch_size=BATCH_SIZE,shuffle=True, seed=seed)
y_val=mask_datagen_val.flow(Y_train[int(Y_train.shape[0]*0.9):],batch_size=BATCH_SIZE,shuffle=True, seed=seed)

In [0]:
#creating a training and validation generator that generate masks and images
train_generator = zip(x, y)
val_generator = zip(x_val, y_val)

In [0]:
# Check if images/masks fit
from matplotlib import pyplot as plt
%matplotlib inline

imshow(x.next()[0].astype(np.uint8))
plt.show()
imshow(np.squeeze(y.next()[0].astype(np.uint8)))
plt.show()
imshow(x_val.next()[0].astype(np.uint8))
plt.show()
imshow(np.squeeze(y_val.next()[0].astype(np.uint8)))
plt.show()

# Train model
Train model with early stopping if loss does not improve for a number of epochs. Additional callbacks to record the loss history and to save best model weights.

In [0]:
# callback to record loss history
# https://keras.io/callbacks/#example-recording-loss-history
class LossHistory(Callback):
    def on_train_begin(self, logs={}):
        self.losses = []
        self.val_losses = []

    def on_batch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))

In [0]:
history = LossHistory()

In [0]:
# upload model weights 
#from google.colab import files
#uploaded = files.upload()

# save as file into current working directory
#with open("model-1.h5", "wb") as f:
#  f.write(uploaded["model-1.h5"])
  
# verify model weights have been uploaded into current working directory
#!ls -l

In [0]:
# load saved model weigths
model = load_model('model-1.h5', custom_objects={'mean_iou': mean_iou})

In [0]:
# fit model using realtime data augmentation
earlystopper = EarlyStopping(patience=15, verbose=1)
checkpointer = ModelCheckpoint('model-1.h5', verbose=1, save_best_only=True)
results = model.fit_generator(train_generator, validation_data=val_generator, validation_steps=10, steps_per_epoch=250,
                              epochs=100, callbacks=[earlystopper, checkpointer, history])

In [0]:
# download model weights
from google.colab import files
files.download('model-1.h5')

## Plot loss history
https://machinelearningmastery.com/display-deep-learning-model-training-history-in-keras/

In [0]:
# summarize history for loss
plt.plot(history.losses)
plt.plot(history.val_losses)
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Predict on train, val and test
Let's make predictions both on the test set, the val set and the train set (as a sanity check). Remember to load the best saved model if you've used early stopping and checkpointing.

In [0]:
# Predict on train, val and test
model = load_model('model-1.h5', custom_objects={'mean_iou': mean_iou})
preds_train = model.predict(X_train[:int(X_train.shape[0]*0.9)], verbose=1)
preds_val = model.predict(X_train[int(X_train.shape[0]*0.9):], verbose=1)
preds_test = model.predict(X_test, verbose=1)

# Threshold predictions
preds_train_t = (preds_train > 0.5).astype(np.uint8)
preds_val_t = (preds_val > 0.5).astype(np.uint8)
preds_test_t = (preds_test > 0.5).astype(np.uint8)

# Create list of upsampled test masks
preds_test_upsampled = []
for i in range(len(preds_test)):
    preds_test_upsampled.append(resize(np.squeeze(preds_test[i]), 
                                       (sizes_test[i][0], sizes_test[i][1]), 
                                       mode='constant', preserve_range=True))

In [0]:
# Perform a sanity check on some random training samples
ix = random.randint(0, len(preds_train_t))
imshow(X_train[ix])
plt.show()
imshow(np.squeeze(Y_train[ix]))
plt.show()
imshow(np.squeeze(preds_train_t[ix]))
plt.show()

In [0]:
# Perform a sanity check on some random validation samples
ix = random.randint(0, len(preds_val_t))
imshow(X_train[int(X_train.shape[0]*0.9):][ix])
plt.show()
imshow(np.squeeze(Y_train[int(Y_train.shape[0]*0.9):][ix]))
plt.show()
imshow(np.squeeze(preds_val_t[ix]))
plt.show()

In [0]:
# show random sample for test dataset and predicted mask 
ix = random.randint(0, len(preds_test_t))
imshow(np.squeeze(X_test[ix]))
plt.show()
imshow(np.squeeze(preds_test[ix]))
plt.show()
imshow(np.squeeze(preds_test_t[ix]))
plt.show()

# Encode and submit results to kaggle
Code for run length encoding from https://www.kaggle.com/rakhlin/fast-run-length-encoding-python

In [0]:
# Run-length encoding
def rle_encoding(x):
    dots = np.where(x.T.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def prob_to_rles(x, cutoff=0.5):
    lab_img = label(x > cutoff)
    for i in range(1, lab_img.max() + 1):
        yield rle_encoding(lab_img == i)

Let's iterate over the test IDs and generate run-length encodings for each seperate mask identified by skimage ...


In [0]:
new_test_ids = []
rles = []
for n, id_ in enumerate(test_ids):
    rle = list(prob_to_rles(preds_test_upsampled[n]))
    rles.extend(rle)
    new_test_ids.extend([id_] * len(rle))

Create submission.

In [0]:
# Create submission DataFrame
sub = pd.DataFrame()
sub['ImageId'] = new_test_ids
sub['EncodedPixels'] = pd.Series(rles).apply(lambda x: ' '.join(str(y) for y in x))
sub.to_csv('results.csv', index=False) # save csv locally

In [0]:
# create results.csv on the client side
# https://stackoverflow.com/questions/31893930/download-csv-from-an-ipython-notebook
from IPython.display import Javascript
js_download = """
var csv = '%s';

var filename = 'results.csv';
var blob = new Blob([csv], { type: 'text/csv;charset=utf-8;' });
if (navigator.msSaveBlob) { // IE 10+
    navigator.msSaveBlob(blob, filename);
} else {
    var link = document.createElement("a");
    if (link.download !== undefined) { // feature detection
        // Browsers that support HTML5 download attribute
        var url = URL.createObjectURL(blob);
        link.setAttribute("href", url);
        link.setAttribute("download", filename);
        link.style.visibility = 'hidden';
        document.body.appendChild(link);
        link.click();
        document.body.removeChild(link);
    }
}
""" % sub.to_csv(index=False).replace('\n','\\n').replace("'","\'")

Javascript(js_download)